In [ ]:
#Import modules, we have to import both pipistrello and iris:
import pipistrello
#import iris

In [ ]:
database_dir = ('/home/juan/MHPC-Thesis/NetCDF_Files')
all_cubes = pipistrello.database(database_dir,new_catalogue=True)
cubesA = all_cubes.load_cubes('')

In [ ]:
from iris.util import describe_diff
from iris.util import unify_time_units
from iris.experimental.equalise_cubes import equalise_attributes

def make_cubes_compatible(list_of_cubes):
    equalise_attributes(list_of_cubes)
    unify_time_units(list_of_cubes)

    for cube_i in list_of_cubes:
        cube_i.cell_methods = ()
    
    c = 0
    for i in range(len(list_of_cubes)):
        for j in range(i+1,len(list_of_cubes)):
            if( not list_of_cubes[i].is_compatible(list_of_cubes[j])):
                print('cubes {} and {}:\n'.format(i,j))
                describe_diff(list_of_cubes[i],list_of_cubes[j])
                c+=1
    if c == 0:
        print("All cubes are now compatible.")
    else:
        print("{} incompatible cubes".format(c))
        
                
make_cubes_compatible(cubesA)

In [ ]:
#cheating: change height of cube 11:
#cubesA[11].coord('height').points = [1.5]
#for i in range(len(cubesA)):
#    height_i = cubesA[i].coord('height')
#    print(i,height_i.points)

def print_span(some_cube):
    for i in some_cube.coords():
        coord_name = i.name()
        if coord_name.lower() == 'time':
            print("{} < {}, {} calendar < {}".format
                  (
                  i.units.num2date(i.points[0]),
                  coord_name, i.units.calendar,
                  i.units.num2date(i.points[-1])
                  )
                 )
        else:
            print("{} < {}, {} < {}".format
                  (
                  i.points[0],
                  coord_name, i.units,
                  i.points[-1]
                  )
                 )


A_concat = cubesA.concatenate()
#print("A=\n{}".format(cubesA[0]))
#print("A=\n{}".format(A_concat[0]))
for cube in A_concat:
    print_span(cube)
    print("")

In [ ]:
test_cube = A_concat[0]

In [ ]:
from iris.time import PartialDateTime
from iris import Constraint
from iris import FUTURE

def get_truth_value(a,years,months,days,hours):

    year_bool = (a.point.year in years) or (years == [])            
    month_bool = (a.point.month in months) or (months == [])
    day_bool = (a.point.day in days) or (days == [])
    hour_bool = (a.point.hour in hours) or (hours == [])
    
    the_constraint = year_bool and month_bool and day_bool and hour_bool
    
    return the_constraint

def pick_times(some_cube,years,months,days,hours):
    tconstraint = Constraint(time = lambda a: get_truth_value(a, years, months, days,hours))

    with FUTURE.context(cell_datetime_objects=True):
        extracted = some_cube.extract(tconstraint)
        if extracted is None:
            t_coord = some_cube.coord('time')
            print("No cube extracted, returning 'None'.")
            print("Is your selection within the time bounds of the original cube?")
            print(t_coord.units.num2date(t_coord.points[0]))
            print(t_coord.units.num2date(t_coord.points[-1]))
            print(t_coord.units.calendar)
            
        return some_cube.extract(tconstraint)

In [ ]:
from iris.analysis.cartography import area_weights
from iris.analysis import MEAN

month_name = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',
              7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}

collapsed_cubes = []
for month in range(1,13):
    print("Getting {}...".format(month_name[month]))
    #1. We extract a cube:
    cube_i = pick_times(test_cube,[],[month],[],[])

    #2. We get the area weights of the cells composing the region:
    grid_areas = area_weights(cube_i)

    #3. We "collapse" our 2D+Time cube into a 0D+Time by averaging using MEAN aggregator:
    collapsed_cubes.append(cube_i.collapsed(['longitude', 'latitude'], MEAN, weights=grid_areas))

print("Done")

In [ ]:
import matplotlib.pyplot as plt
import iris.quickplot as iplt

for i in range(len(collapsed_cubes)):
    #Plot...
    print("Plotting {}...".format(month_name[i+1]))
    iplt.plot(collapsed_cubes[i],linewidth='10',label=month_name[i+1])
    plt.legend(loc=4)
    #iplt.plot(c_interp,'b.')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import iris.quickplot as iplt
import sys
import os, errno
import utils as ui
import matplotlib.pyplot as plt
import iris
import iris.quickplot as iplt
import numpy as np

from __future__ import (absolute_import, division, print_function)
from six.moves import (filter, input, map, range, zip)  # noqa
